# 2-4 ネットワークモデルの実装

## SSD ネットワークモデルの概要
SSD のネットワークモデルの概要を次に示す．

<img src="../image/IMG_0194.jpg" alt="structure_ssd_network">

前処理を行った 300 x 300 の画像データを入力し，8,732個のデフォルトボックスに対するオフセット情報と21種類のクラスに対する信頼度を出力として得る．

vgg モジュールに入力された画像は VGG-16 モデルをベースにしたネットワークを通過する．
畳み込み層のカーネルサイズや使用するユニットは同じだが，各ユニットの出力する特徴量マップのサイズは VGG-16 とは異なる．
vgg モジュールで畳み込み層を10回通過したデータ（conv4_3）を抜き出し，L2 正規化を施して source1 とする．
この source1 はチャネル数512，38 x 38 の特徴量マップである．
また，vgg モジュールの最終的な出力を source2 とする．
source2 はチャネル数1,024，19 x 19 の特徴量マップである．

次に，source2 を extras モジュールに入力する．
ここでは，8回の畳み込みを行い2回ごとに特徴量マップを source3~6 として出力する．
それぞれの source のサイズは10x10，5x5，3x3，1x1となっている．
ここで重要になるのは，source の各特徴量マップがそれぞれ異なっていることである．
source1 では 38x38=1444 の領域について，それぞれの特徴量マップから物体を検出しようとしている一方で，source6　は画像全体を1つの大きな特徴量として捉え，大きな物体を検出しようとしている．
ただし，source によって畳み込みの回数が異なるため検出能力に差が出てくる．
例えば，source1 は10回の畳み込みしか通過していないのに対して，source6 は23回の畳み込みを受けている．
そのため，小さな領域の特徴量を十分に捉えきれず，SSD は小さな物体の検出には向かないという欠点がある．

こうして得られた source は loc モジュールと conf モジュールに入力され，それぞれからデフォルトボックスに対するオフセット情報と，各クラスに対する信頼度を出力する．
各 souce から得られる特徴量マップは合計で1,940個だが，source1, 5, 6 に対して4個，source2, 3, 4 に対しては6個のデフォルトボックスを用意している．
そのため，デフォルトボックスの合計は8,732個となる

## モジュール vgg を実装
vgg モジュールを実装する関数 make_vgg を作成する．
畳み込み層，ReLU 関数，マックスプーリングを合計で34ユニット使用する．
各畳み込み層のチャネル数とマックスプーリングの情報を cfg = $[64, 64, 'M', ・・・]$ として変数に格納しておく．
’M'はマックスプーリングを，’MC'はマックスプーリングを ceil モードで計算することを表している．
なお，ReLU の引数 inplace を True としておくと，ReLU への入力をメモリ上に保持せず上書きするため，メモリの節約になる．

In [18]:
import torch
import pandas as pd
from torch import nn
from numpy import sqrt
from itertools import product

In [2]:
def make_vgg():
    ''' 34層の VGG モジュールを作成する '''
    layers = []
    in_channels = 3  # 入力の色チャネル
    
    # VGG モジュールで使う畳み込み層やマックスプーリングのチャネル数
    # M  : マックスプーリング（出力テンソルのサイズは floor モード）
    # MC : マックスプーリング（出力テンソルのサイズは ceil モード）
    cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'MC', 512, 512, 512, 'M', 512, 512, 512]
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'MC':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
            
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6, nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    
    return nn.ModuleList(layers)

In [3]:
# 動作確認
vgg_test = make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

## モジュール extras を実装
8個の畳み込みユニットを並べて extras モジュールを実装する make_extras 関数を実装する

In [4]:
# 8層の extras モジュールを作成
def make_extras():
    layers =[]
    in_channels = 1024
    
    # extras モジュールの畳み込み層のチャネル数を設定するコンフィギュレーション
    cfg = [256, 512, 128, 256, 128, 256, 128, 256]
    
    layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))]
    layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))]
    
    return nn.ModuleList(layers)

In [5]:
# 動作確認
extras_test = make_extras()
print(extras_test)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


## モジュール loc とモジュール conf を実装
loc と conf の各モジュールでは6つの畳み込み層が用意されているが，6つの畳み込み層を前から後ろへ順伝播するのではなく，それぞれの source に対応して1回ずつ計算される．
順伝播の実装は次節で定義する．

In [6]:
def make_loc_conf(num_classes=21, bbox_aspect_num=[4, 6, 6, 6, 4, 4]):
    loc_layers = []
    conf_layers = []
    
    # VGG の22層目，conv4_3（source1）に対する畳み込み層
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[0] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[0] * num_classes, kernel_size=3, padding=1)]
    # VGG の最終層（source2）に対する畳み込み層
    loc_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * num_classes, kernel_size=3, padding=1)]
    # extras（source3）に対する畳み込み層
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[2] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[2] * num_classes, kernel_size=3, padding=1)]
    # extras（source4）に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[3] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[3] * num_classes, kernel_size=3, padding=1)]
    # extras（source5）に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[4] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[4] * num_classes, kernel_size=3, padding=1)]
    # extras（source6）に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[5] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[5] * num_classes, kernel_size=3, padding=1)]
    
    return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)

In [7]:
#動作確認
loc_test, conf_test = make_loc_conf()
print(loc_test)
print(conf_test)

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


## L2Norm 層を実装
L2Norm 層ではチャネルごとに特徴量マップが異なる点を正規化する．
ここでは，1,444個の特徴量マップのセルごとに2乗和を計算し，その平方根で割り算して正規化を行う．
さらに，512x38x38のテンソルに対して係数をかける．
この512個の係数は学習パラメータである．

In [27]:
# conv4_3 からの出力を scale=20 の L2Norm で正規化する層
class L2Norm(nn.Module):
    def __init__(self, input_cannels=512, scale=20):
        super(L2Norm, self).__init__()  # 親クラスのコンストラクタを実行
        self.weight = nn.Parameter(torch.Tensor(input_cannels))
        self.scale = scale  # 係数 weight の初期値として設定する値
        self.reset_parameters() # パラメータの初期化
        self.eps = 1e-10
        
    def reset_parameters(self):
        ''' 結合パラメータを大きさ scale の値にする初期化を実行 '''
        nn.init.constant_(self.weight, self.scale)  # weight の値が全て scale になる
        
    def forward(self, x):
        """
        38x38 の特徴量に対して512チャネルについて二乗和の平方根を求めた 38x38 個の値を使用し，
        各特徴量を正規化してから係数を掛け算する
        """
        
        # 各チャネルについて正規化，テンソルサイズは torch.Size([batch_num, 1, 38, 38])
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt() + self.eps
        x = torch.div(x, norm)
        
        # 係数（torch.Size([512])）を掛ける
        # torch.Size([batch_num, 512, 38, 38]) のテンソルに変形
        weights = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
        out = weights * x
        
        return out

## デフォルトボックスを実装
それぞれの source に対して4個または6個のデフォルトボックスを作成する．
デフォルトボックスが4つの場合，小さい正方形，大きい正方形，1:2の長方形，2:1の長方形で，6種類の場合にはこれに加えて3:1と1:3の長方形を用意する．

In [21]:
# デフォルトボックスを出力するクラスの実装
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()
        
        # 初期設定
        self.image_size = cfg['input_size']        # 画像サイズ 300
        self.feature_maps = cfg['feature_maps']    # [38, 19, ...] 各 source も特徴量マップのサイズ
        self.num_priors = len(cfg["feature_maps"]) # source の個数 6
        self.steps = cfg['steps']                  # [8, 16, ...] デフォルトボックスのピクセルサイズ
        self.min_sizes = cfg['min_sizes']          # [30, 60, ...] 小さい正方形のデフォルトボックスのピクセルサイズ
        self.max_sizes = cfg['max_sizes']          # [60, 111, ...] 大きい正方形のデフォルトボックスのピクセルサイズ
        self.aspect_ratios = cfg['aspect_ratios']  # 長方形のデフォルトボックスのアスペクト比
        
    def make_dbox_list(self):
        ''' デフォルトボックスの作成 '''
        mean = []
        # 'feature_maps' : [38, 19, 10, 5, 3, 1]
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):  # f までの数で2ペアの組み合わせを作る f_P_2
                
                # 特徴量の画像サイズ
                # 300 / 'steps' : [8, 16, 32, 64, 100, 300]
                f_k = self.image_size / self.steps[k]
                
                # デフォルトボックスの中心座標 x, y ただし 0~1 で規格化している
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k
                
                # アスペクト比 1 の小さいデフォルトボックス [cx, cy, width, height]
                # 'min_sizes': [30, 60, 111, 162, 213, 264]
                s_k = self.min_sizes[k] / self.image_size
                mean += [cx, cy, s_k, s_k]
                
                # アスペクト比 1 の大きいデフォルトボックス [cx, cy, width, height]
                # 'min_sizes': [45, 99, 153, 207, 261, 315]
                s_k_prime = sqrt(s_k * (self.max_sizes[k] / self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                # その他のアスペクト比の defBox [cx, cy, width, height]
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k * sqrt(ar), s_k / sqrt(ar)]
                    mean += [cx, cy, s_k / sqrt(ar), s_k * sqrt(ar)]
                
        # デフォルトボックスをテンソルに変換 torch.Size([8372, 4])
        output = torch.Tensor(mean).view(-1, 4)
        # デフォルトボックスが画像外にはみ出すのを防ぐため大きさを最大1，最小0にする
        output.clamp_(max=1, min=0)
        
        return output

In [22]:
# 動作確認
# SSD300 の設定
ssd_cfg = {
'num_classes': 21,                          # 背景クラスを含めた合計クラス数
'input_size': 300,                          # 画像の入力サイズ
'bbox_aspect_num': [4, 6, 6, 6, 4, 4],      # 出力する デフォルトボックス のアスペクト比の種類
'feature_maps': [38, 19, 10, 5, 3, 1],      # 各 source の画像サイズ
'steps': [8, 16, 32, 64, 100, 300],         # デフォルトボックス の大きさを決める
'min_sizes': [30, 60, 111, 162, 213, 264],  # デフォルトボックス の大きさを決める
'max_sizes': [60, 111, 162, 213, 264, 315], # デフォルトボックス の大きさを決める
'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}
# デフォルトボックスを作成
dbox = DBox(ssd_cfg)
dbox_list = dbox.make_dbox_list()
# デフォルトボックスの出力を確認する
pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
5,0.040000,0.013333,0.141421,0.141421
6,0.040000,0.013333,0.141421,0.070711
7,0.040000,0.013333,0.070711,0.141421
8,0.066667,0.013333,0.100000,0.100000
9,0.066667,0.013333,0.141421,0.141421


## クラス SSD を実装する
最後に，PyTorch のネットワーク層クラス nn.Module を継承し SSD クラスを実装する．
ただし，順伝播のメソッドは次節で追加する．

In [25]:
# SSD クラスを実装
class SSD(nn.Module):
    def __init__(self, phase, cfg):
        super(SSD, self).__init__()
        self.phase = phase  #train or inference
        self.num_classes = cfg["num_classes"]  # クラス数 21
        
        # SSD のネットワークを作る
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg["num_classes"], cfg["bbox_aspect_num"])
        
        # デフォルトボックスの作成
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        # 推論時はクラス「Detect」を用意
        if phase == 'inference':
            self.detec = Detect()

In [26]:
# 動作確認
ssd_test = SSD(phase="train", cfg=ssd_cfg)
print(ssd_test)

NameError: name 'init' is not defined